### 직방의 원룸 매물정보 수집
- 절차
    - 동이름 > 위도, 경도
    - 위도, 경도 > geohash 변환
    - 매물 아이디 > 매물 정보

In [72]:
!pip install geohash2

In [73]:
import requests
import pandas as pd
import geohash2

In [74]:
# 1. 동이름 > 위도, 경도

In [75]:
# payload 깨질때 쓰는 사이트
# https://meyerweb.com/eric/tools/dencoder/

In [76]:
addr = "강남구 역삼동"
url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸"

response = requests.get(url)
data = response.json()["items"][0]
lat, lng = data["lat"], data["lng"]
lat, lng

(37.495365142822266, 127.03306579589844)

In [77]:
# 2. 위도, 경도 > geohash

In [78]:
# precision의 커질수록 영역이 작아짐
geohash = geohash2.encode(lat, lng, precision=5) # precision default 값 : 12
geohash

'wydm6'

In [79]:
# 3. geohash > 매물 아이디

In [80]:
url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸"
url

'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash=wydm6&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'

In [81]:
response = requests.get(url)
items = response.json()["items"]
item_id = [item["item_id"] for item in items]
item_id[:5]

[32819635, 32838798, 32872608, 32838700, 32767020]

In [82]:
# 4. 매물 아이디 > 매물 정보

In [83]:
url = "https://apis.zigbang.com/v2/items/list"

params = {
    "domain": "zigbang",
    "withCoalition": "true",
    "item_ids": item_id[:900]
}

response = requests.post(url, data=params)
response

<Response [200]>

In [84]:
items = response.json()["items"]
items[0]

{'section_type': None,
 'item_id': 32819635,
 'images_thumbnail': 'https://ic.zigbang.com/ic/items/32819635/1.jpg',
 'sales_type': '전세',
 'sales_title': '전세',
 'deposit': 16500,
 'rent': 0,
 'size_m2': 39.89,
 '공급면적': {'m2': 39.89, 'p': '12.1'},
 '전용면적': {'m2': 20.82, 'p': '6.3'},
 '계약면적': None,
 'room_type_title': None,
 'floor': '중',
 'floor_string': '중',
 'building_floor': '20',
 'title': '★전입신고O★10월14일이후★채광없음★일반전세자금대출가능★',
 'is_first_movein': None,
 'room_type': '01',
 'address': '서초구 서초동',
 'random_location': {'lat': 37.48566359818854, 'lng': 127.0136073917663},
 'is_zzim': False,
 'status': True,
 'service_type': '오피스텔',
 'tags': [],
 'address1': '서울시 서초구 서초동',
 'address2': None,
 'address3': None,
 'manage_cost': '7.5',
 'reg_date': '2022-07-30T13:43:13+09:00',
 'is_new': False}

In [85]:
# max row, max column 설정
# pd.options.display.max_columns, pd.options.display.max_rows

In [86]:
pd.options.display.max_columns = 50

In [87]:
pd.options.display.max_rows = 50

In [88]:
df = pd.DataFrame(items)
display(df.tail(2))

,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,계약면적,room_type_title,floor,floor_string,building_floor,title,is_first_movein,room_type,address,random_location,is_zzim,status,service_type,tags,address1,address2,address3,manage_cost,reg_date,is_new
898,None,32844154,https://ic.zigbang.com/ic/items/32844154/1.jpg,월세,월세,100,100,29.75,"{'m2': 29.75, 'p': '9'}","{'m2': 29.75, 'p': '9'}",None,None,2,2,2,🟥🟦단기💖맛집🟦🟥 리모델링 주거/업무 겸용 원룸!,None,01,강남구 역삼동,"{'lat': 37.5016794314628, 'lng': 127.029515652...",False,True,원룸,[추천],서울시 강남구 역삼동,None,None,9,2022-08-01T18:43:35+09:00,False
899,None,32656329,https://ic.zigbang.com/ic/items/32656329/1.jpg,전세,전세,24000,0,59.50,"{'m2': 59.5, 'p': '18'}","{'m2': 34.71, 'p': '10.5'}",None,None,중,중,26,"◈전입가능◈위치,크기좋은 전세◈",None,01,강남구 역삼동,"{'lat': 37.50222235548125, 'lng': 127.04454226...",False,True,오피스텔,[추천],서울시 강남구 역삼동,None,None,10,2022-07-18T13:23:18+09:00,False


In [89]:
columns = ["item_id", "sales_type", "deposit", "rent", "size_m2", "address1", "manage_cost"]
df = pd.DataFrame(items)[columns]
display(df)

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
0,32819635,전세,16500,0,39.89,서울시 서초구 서초동,7.5
1,32838798,월세,1000,68,37.56,서울시 서초구 서초동,7.5
2,32872608,월세,1000,70,37.56,서울시 서초구 서초동,7.5
3,32838700,전세,50400,0,66.12,서울시 서초구 서초동,9
4,32767020,월세,20000,90,60.12,서울시 서초구 서초동,5
...,...,...,...,...,...,...,...
895,32850137,월세,300,260,32.91,서울시 강남구 역삼동,10
896,32852414,월세,130,250,33.44,서울시 강남구 역삼동,10
897,32809154,월세,120,70,19.83,서울시 강남구 역삼동,9
898,32844154,월세,100,100,29.75,서울시 강남구 역삼동,9


In [90]:
# 함수 만들기
def oneroom(addr):
    url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸"

    response = requests.get(url)
    data = response.json()["items"][0]
    lat, lng = data["lat"], data["lng"]
    
    geohash = geohash2.encode(lat, lng, precision=5) # precision default 값 : 12
    
    url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸"
    response = requests.get(url)
    items = response.json()["items"]
    item_id = [item["item_id"] for item in items]
    
    url = "https://apis.zigbang.com/v2/items/list"

    params = {
        "domain": "zigbang",
        "withCoalition": "true",
        "item_ids": item_id[:900]
    }

    response = requests.post(url, data=params)
    items = response.json()["items"]
    columns = ["item_id", "sales_type", "deposit", "rent", "size_m2", "address1", "manage_cost"]
    
    return pd.DataFrame(items)[columns]

In [91]:
addr = "마포구 합정동"
df = oneroom(addr)
df.tail(2)

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
95,32812101,월세,3000,80,61.24,서울시 마포구 합정동,2
96,32836233,월세,1000,45,13.95,서울시 마포구 합정동,5


In [92]:
df_filtered = df[df["address1"].str.contains(addr)].reset_index(drop=True)
df_filtered

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
0,32662713,전세,17700,0,39.67,서울시 마포구 합정동,0
1,32668759,전세,17700,0,39.67,서울시 마포구 합정동,0
2,32726690,전세,19900,0,36.28,서울시 마포구 합정동,0
3,32542700,월세,1500,95,49.59,서울시 마포구 합정동,7
4,32513282,전세,5000,0,23.14,서울시 마포구 합정동,5
...,...,...,...,...,...,...,...
92,32562277,월세,1000,90,62.00,서울시 마포구 합정동,2
93,32769021,월세,1000,90,62.81,서울시 마포구 합정동,2
94,32804404,월세,2000,85,61.24,서울시 마포구 합정동,2
95,32812101,월세,3000,80,61.24,서울시 마포구 합정동,2


In [93]:
# 모듈 파일 만들기 : py

In [94]:
%ls

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: D3A3-08A0

 C:\Users\Wslee\Desktop\github\aivle\3. web crawling\day2 디렉터리

2022-08-04  오후 01:41    <DIR>          .
2022-08-04  오후 01:41    <DIR>          ..
2022-08-04  오후 01:39    <DIR>          .ipynb_checkpoints
2022-08-04  오후 01:35    <DIR>          __pycache__
2022-08-04  오전 11:05            54,726 01_.ipynb
2022-08-04  오후 01:39            41,091 02_zigbang.ipynb
2022-08-04  오후 01:41             1,099 03_daum_exchange.ipynb
2022-08-04  오후 01:41             1,171 zigbang.py
               4개 파일              98,087 바이트
               4개 디렉터리  44,630,360,064 바이트 남음


In [157]:
%%writefile zigbang.py
import requests
import pandas as pd
import geohash2

def oneroom(addr):
    url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={addr}&serviceType=원룸"

    response = requests.get(url)
    data = response.json()["items"][0]
    lat, lng = data["lat"], data["lng"]
    
    geohash = geohash2.encode(lat, lng, precision=5) # precision default 값 : 12
    
    url = f"https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang\
&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸"
    response = requests.get(url)
    items = response.json()["items"]
    item_id = [item["item_id"] for item in items]
    
    url = "https://apis.zigbang.com/v2/items/list"

    params = {
        "domain": "zigbang",
        "withCoalition": "true",
        "item_ids": item_id[:900]
    }

    response = requests.post(url, data=params)
    items = response.json()["items"]
    columns = ["item_id", "sales_type", "deposit", "rent", "size_m2", "address1", "manage_cost"]
    addr = "마포구 합정동"
    df =pd.DataFrame(items)[columns]
    df_filtered = df[df["address1"].str.contains(addr)].reset_index(drop=True)
    return df_filtered

Overwriting zigbang.py


In [158]:
import zigbang as zb

In [159]:
addr = "마포구 합정동"
df = zb.oneroom(addr)
df_filtered = df[df["address1"].str.contains(addr)].reset_index(drop=True)
display(df_filtered.head(100))

,item_id,sales_type,deposit,rent,size_m2,address1,manage_cost
0,32662713,전세,17700,0,39.67,서울시 마포구 합정동,0
1,32668759,전세,17700,0,39.67,서울시 마포구 합정동,0
2,32726690,전세,19900,0,36.28,서울시 마포구 합정동,0
3,32542700,월세,1500,95,49.59,서울시 마포구 합정동,7
4,32513282,전세,5000,0,23.14,서울시 마포구 합정동,5
...,...,...,...,...,...,...,...
92,32562277,월세,1000,90,62.00,서울시 마포구 합정동,2
93,32769021,월세,1000,90,62.81,서울시 마포구 합정동,2
94,32804404,월세,2000,85,61.24,서울시 마포구 합정동,2
95,32812101,월세,3000,80,61.24,서울시 마포구 합정동,2
